In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

Classe para os nós da árvore com atributos:

*attribute : atributo que vai ser avaliado no nó (não existe se o nó for folha);

*prev_value : valor que o ramo que levou ao nó tem, ou seja, o valor da atributo do nó anterior (não existe na raiz da árvore);

*classification : 'Class' do nó, ou seja, é a classificação final pretendida (só existe nas folhas da árvore);

*children : filhos do nó (não existe nas folhas da árvore);

In [3]:
class TreeNode:
    def __init__(self, attribute=None, prev_value=None, classification=None, children=None) -> None:
        self.attribute = attribute #atributo a ser avaliado no nó SE NÃO FOR FOLHA
        self.prev_value = prev_value #valor do atributo do nó pai, como se fosse o valor do ramo SE NÃO FOR A RAIZ
        self.classification = classification #classificação da classe final SE FOR FOLHA
        self.children = children if children is not None else []

    def copy(self):
        return deepcopy(self)

    def __str__(self) -> str:
        return str(self.attribute) + ' / ' + str(self.prev_value) + ' / ' + str(self.classification)
        
    def add_child(self, child) -> None:
        self.children.append(child)

    def is_leaf(self) -> bool:
        return len(self.children) == 0
    
    def add_children(self, children=[]) -> None:
        self.children += children
        
    def print_tree(self, level=0) -> None:
        prefix = "|   " * level
        print(f"{prefix}|-- {self.attribute} |Valor do ramo:{self.prev_value if self.prev_value is not None else '##'}|  |Classe:{self.classification if self.classification is not None else '##'}|")
        for child in self.children:
            child.print_tree(level + 1)

Função para calcular a entropia de um atributo do DataSet.

In [4]:
def entropy(df: pd.DataFrame, attribute: str) -> float:
    if len(df) == 0 or len(df[attribute].unique()) == 1:
        return 0 
    # Implementação vetorizada para aumentar eficiencia 
    # normalize = true faz a proporção dos valores 
    value_counts = df[attribute].value_counts(normalize=True)
    entropy = -(value_counts * np.log2(value_counts)).sum()
    return entropy

Função para calcular a entropia condicional.

In [5]:
def conditional_entropy(df: pd.DataFrame, attribute: str, target_attribute: str) -> float: #H(attribute | target_attribute)
    target_classes = df[target_attribute].unique()
    entropy_attribute = 0
    total_examples = len(df)

    for cls in target_classes:
        subset_df = df[df[target_attribute] == cls]
        entropy_subset = entropy(subset_df, attribute)
        prob_cls = len(subset_df) / total_examples #probabilidade desta classe acontecer P(cls)
        entropy_attribute += prob_cls * entropy_subset
    return entropy_attribute

Função para o cálculo do ganho de informação.

In [6]:
def information_gain(df: pd.DataFrame, attribute: str) -> float: 
    return entropy(df, 'Class') - conditional_entropy(df, attribute, 'Class')

Função para saber qual o atributo da lista que fornece maior ganho de informação.

In [7]:
def max_gain(df: pd.DataFrame, attributes: list) -> str:
    max_info_gain = (None, float('-inf'))
    for attribute in attributes:
        gain = information_gain(df, attribute)
        if gain > max_info_gain[1]:
            max_info_gain = (attribute, gain)
    return max_info_gain[0]

Função para saber qual o valor mais comum da 'Class'.

In [8]:
def plurality_value(df: pd.DataFrame) -> int:
    values = df['Class'].value_counts().to_dict()
    max_value = (None, float('-inf'))
    for key, value in values.items():
        if value > max_value[1]:
            max_value = (key, value)
    return max_value[0]

Função para construir a árvore de decisão.

In [9]:
def decisionTree(original_df: pd.DataFrame, df: pd.DataFrame, attributes: list, parent_df=None) -> TreeNode:
    
    if df.empty: #sem exemplos
        return TreeNode(classification= plurality_value(parent_df))

    if len(df['Class'].unique()) == 1: #todos os exemplos têm a mesma classe
        return TreeNode(classification= df['Class'].iloc[0])

    if len(attributes) == 0: #sem atributos
        return TreeNode(classification= plurality_value(df))

    attribute = max_gain(df, attributes) #atributo mais importante
    tree = TreeNode(attribute= attribute) #decisionTree
    
    attributes.remove(attribute) #remover atributo com max info gain para a recursão
    
    possible_values = set(original_df[attribute].values) #todos os possiveis valores do atributo com max info gain
    
    for value in possible_values:
        sub_df = df[df[attribute] == value] #exemplos do atributo com v=value
        subtree = decisionTree(original_df= original_df, df= sub_df, attributes= attributes.copy(), parent_df= df)
        subtree.prev_value = value
        tree.add_child(subtree)
    return tree
